In [8]:
import yfinance as yf
import pandas as pd
!pip install backtesting

# List of 20 US stocks and ETFs
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META',
    'TSLA', 'NVDA', 'JPM', 'JNJ', 'V',
    'SPY', 'QQQ', 'DIA', 'IWM', 'ARKK',
    'XLF', 'XLK', 'XLV', 'XLE', 'XLY'
]

# Download 10 years of daily data
data = yf.download(tickers, start="2015-01-01", end="2025-01-01", group_by='ticker', auto_adjust=False)

[*********************100%***********************]  20 of 20 completed


In [9]:
# Function to calculate streak metrics based on candle colors
def count_candle_streaks(n, data):
    up_streaks = 0
    up_followed_by_red = 0
    down_streaks = 0
    down_followed_by_green = 0

    for ticker in tickers:
        df = data[ticker][['Open', 'Close']].dropna()
        candle_color = df['Close'] > df['Open']  # True: green, False: red
        candle_series = candle_color.values

        i = 0
        while i < len(candle_series) - n:
            # Check for green streak
            if all(candle_series[i + j] for j in range(n)):
                up_streaks += 1
                if i + n < len(candle_series) and not candle_series[i + n]:
                    up_followed_by_red += 1
                i += n
            # Check for red streak
            elif all(not candle_series[i + j] for j in range(n)):
                down_streaks += 1
                if i + n < len(candle_series) and candle_series[i + n]:
                    down_followed_by_green += 1
                i += n
            else:
                i += 1

    up_fraction = up_followed_by_red / up_streaks if up_streaks > 0 else 0
    down_fraction = down_followed_by_green / down_streaks if down_streaks > 0 else 0

    return {
        f"{n} Green Candles Followed by Red (Fraction)": up_fraction,
        f"{n} Red Candles Followed by Green (Fraction)": down_fraction
    }

In [10]:
metrics = count_candle_streaks(10, data)
print(metrics)

{'10 Green Candles Followed by Red (Fraction)': 0.41935483870967744, '10 Red Candles Followed by Green (Fraction)': 0.6538461538461539}


In [11]:
from backtesting import Strategy, Backtest

class ConsecutiveRedGreenStrategy(Strategy):
    """
    Long entry: After n red candles, buy at next open, close at same bar close.
    Short entry: After n green candles, sell at next open, close at same bar close.
    """
    n = 4  # Number of consecutive candles to trigger entry

    def init(self):
        pass

    def next(self):
        if len(self.data) < self.n:
            return

        open_ = self.data.Open
        close_ = self.data.Close

        # LONG ENTRY: After n red candles
        if all(close_[-i] < open_[-i] for i in range(1, self.n + 1)):
            self.buy()

        # SHORT ENTRY: After n green candles
        elif all(close_[-i] > open_[-i] for i in range(1, self.n + 1)):
            self.sell()

        # Always close open positions at the same bar's close
        if self.position:
            self.position.close()


/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [12]:
from backtesting import Strategy, Backtest

class ConsecutiveRedStrategy(Strategy):
    """
    If we detect n red candles in a row, then on the *very next* bar:
      - Buy at its open
      - Close at that same bar's close
    """
    n = 3  # number of consecutive red candles required

    def init(self):
        pass

    def next(self):
        # We need at least n bars of history
        if len(self.data) < self.n:
            return

        # Check if last n candles were red: Close < Open for each of the last n bars
        # The most recent bar is at index -1, then -2, etc.
        if all(self.data.Close[-i] < self.data.Open[-i] for i in range(1, self.n + 1)):
            # Issue a Buy at the *open* of the current bar (index -1),
            # which is effectively "the next bar" from the perspective
            # of completed candles.
            self.buy()

        # If we have an open position, close it at *this* bar's close.
        # Because Backtesting.py processes order fills at the bar open,
        # calling `self.position.close()` inside `next()` ensures the position
        # will be exited by this bar’s close.
        if self.position:
            self.position.close()


In [13]:
import numpy as np

results = {}

for symbol in tickers:
    df = data[symbol][['Open', 'High', 'Low', 'Close', 'Volume']] #.dropna()
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    bt = Backtest(df,
                  ConsecutiveRedGreenStrategy,
                  cash=100_000,
                  commission=0.0004, # 0.002
                  exclusive_orders=True,
                  margin=1/1)

    stats = bt.run()
    results[symbol] = stats

# Extract (symbol, return%) as a list of tuples
returns_list = [(sym, res['Return [%]']) for sym, res in results.items()]

# For demonstration, let's print them
print("Returns for each ticker:")
for sym, ret in returns_list:
    print(f"{sym}: {ret:.2f}%")

# Compute aggregated metrics
sum_of_returns = sum(ret for _, ret in returns_list)
average_return = np.mean([ret for _, ret in returns_list])
max_drawdowns = [(sym, res['Max. Drawdown [%]']) for sym, res in results.items()]
max_drawdown_overall = min(mdd for _, mdd in max_drawdowns)  # or min, depending on how you interpret it

print("\n=== Aggregated Metrics ===")
print(f"Sum of Returns: {sum_of_returns:.2f}%")
print(f"Average Return: {average_return:.2f}%")
print(f"Maximum Drawdown across all tickers: {max_drawdown_overall:.2f}%")

# Optionally, you could gather more stats into a separate dictionary for final summary:
summary_metrics = {
    'Sum of Returns': sum_of_returns,
    'Average Return': average_return,
    'Max Drawdown Overall': max_drawdown_overall
}

print("\nSummary Metrics:", summary_metrics)


Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2515 [00:00<?, ?bar/s]

Returns for each ticker:
AAPL: -10.69%
MSFT: -12.35%
GOOGL: -10.90%
AMZN: 56.78%
META: 71.91%
TSLA: 60.33%
NVDA: -13.42%
JPM: -28.41%
JNJ: 2.32%
V: 77.97%
SPY: -19.96%
QQQ: -1.30%
DIA: -38.14%
IWM: -27.31%
ARKK: -25.21%
XLF: -26.34%
XLK: 1.53%
XLV: -12.28%
XLE: -29.17%
XLY: 11.74%

=== Aggregated Metrics ===
Sum of Returns: 27.13%
Average Return: 1.36%
Maximum Drawdown across all tickers: -46.76%

Summary Metrics: {'Sum of Returns': 27.125991513629753, 'Average Return': np.float64(1.356299575681489), 'Max Drawdown Overall': -46.760489732148805}
